You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20362160262"> Volatility Scatter Plots and Regressions (13 min)</a>

In [14]:
# import the python libraries
import bql 
import pandas as pd
import statsmodels.api as sm
import bqplot as bqp
import datetime 

# instantiate the bql service
bq = bql.Service()

In [15]:
# specify your forward looking period
forward_days = 90

# format this forward looking period as a string
forward_days_string = '{}d'.format(forward_days)

In [16]:
# specify your security
security = 'GLD US Equity'

# create your query for the implied volatility timeseries
implied_vol = bq.func.dropna(bq.data.implied_volatility(dates=bq.func.range('-1y', '0d'), expiry=forward_days_string, delta='50'))

bql_request = bql.Request(security, {'implied vol':implied_vol})
bql_response = bq.execute(bql_request)
impdf = bql_response[0].df()
impdf

,DATE,implied vol
ID,,
GLD US Equity,2019-07-01,12.4183
GLD US Equity,2019-07-02,13.8201
GLD US Equity,2019-07-03,13.9359
GLD US Equity,2019-07-05,13.2159
GLD US Equity,2019-07-08,11.7777
GLD US Equity,2019-07-09,11.9411
GLD US Equity,2019-07-10,13.1561
GLD US Equity,2019-07-11,12.3303
GLD US Equity,2019-07-12,12.8704


In [17]:
# pull realized volatility lagged by the forward period
total_return = bq.data.day_to_day_total_return(dates=bq.func.range('-1y','0d'))

lead_ret = bq.func.first(bq.func.last(bq.func.lead(total_return,forward_days),forward_days*2),forward_days)

fwd_std = bq.func.std(bq.func.dropna(lead_ret))

roll_fwd_std = bq.func.rolling(fwd_std,bq.func.range('-1y','0d'))*bq.func.sqrt(252)

items = {'Realized vol':roll_fwd_std * 100}

volreq = bql.Request(security,items)
volresp = bq.execute(volreq)
realdf = volresp[0].df()
realdf

,DATE,ORIG_IDS,ITERATION_DATE,ITERATION_ID,Realized vol
ID,,,,,
GLD US Equity,2019-03-30,GLD US Equity,2019-06-29,GLD US Equity:ITR0,9.935829
GLD US Equity,2019-03-30,GLD US Equity,2019-06-30,GLD US Equity:ITR1,9.956850
GLD US Equity,2019-04-02,GLD US Equity,2019-07-01,GLD US Equity:ITR2,10.796975
GLD US Equity,2019-04-03,GLD US Equity,2019-07-02,GLD US Equity:ITR3,11.526425
GLD US Equity,2019-04-04,GLD US Equity,2019-07-03,GLD US Equity:ITR4,11.524470
GLD US Equity,2019-04-04,GLD US Equity,2019-07-04,GLD US Equity:ITR5,11.608201
GLD US Equity,2019-04-06,GLD US Equity,2019-07-05,GLD US Equity:ITR6,11.792752
GLD US Equity,2019-04-06,GLD US Equity,2019-07-06,GLD US Equity:ITR7,11.792752
GLD US Equity,2019-04-06,GLD US Equity,2019-07-07,GLD US Equity:ITR8,11.874164


In [18]:
# reformat your implied volatility dataframe
impdf2 = impdf.reset_index().drop('ID',axis=1)

In [19]:
# reformat your realized volatility dataframe
realdf2 = realdf.reset_index().drop(['ORIG_IDS','ITERATION_DATE','ITERATION_ID','ID'],axis=1)

In [20]:
# merge the two dataframes together
dfmerged = pd.merge(realdf2,impdf2,on='DATE').set_index('DATE')
dfmerged

,Realized vol,implied vol
DATE,,
2019-07-02,15.429408,13.8201
2019-07-03,15.458419,13.9359
2019-07-05,15.307373,13.2159
2019-07-09,15.349461,11.9411
2019-07-10,15.138475,13.1561
2019-07-11,15.016993,12.3303
2019-07-12,15.107717,12.8704
2019-07-16,15.056534,12.7604
2019-07-17,14.805435,13.9087


In [21]:
# do the regression
X = dfmerged.dropna()[['implied vol']]
y = dfmerged.dropna()[['Realized vol']]

X = sm.add_constant(X)

model = sm.OLS(exog=X,endog=y).fit()
predictions = model.predict(X)

# specify your line for the visualization
lineintercept = model.params['const']
lineslope = model.params['implied vol']
X_line = [X['implied vol'].min(),X['implied vol'].max()]
y_line = [X_line[0]*lineslope+lineintercept,X_line[1]*lineslope+lineintercept]

In [22]:
# format the index column as a string
dataframe = dfmerged.assign(Date=dfmerged.index.strftime('%Y-%m-%d'))
dataframe_for_scatter = dataframe.dropna()

In [23]:
## This code is for creating the scatterplot with a date colorscale, regression, and enhanced tooltip ##

# Create Scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()
sc_c1 = bqp.DateColorScale()

# Create Mark
tooltip = bqp.Tooltip(fields=['name','x', 'y'],
                      labels=['Date','implied vol', 'realized vol'],
                      formats=['','.3f', '.3f'])
mark_scatter = bqp.Scatter(x=dataframe_for_scatter['implied vol'],
                           y=dataframe_for_scatter['Realized vol'],
                           names = dataframe_for_scatter.index.strftime('%Y-%m-%d'),
                           color = dataframe_for_scatter.index,
                           tooltip=tooltip,
                           scales={'x': scale_x, 'y': scale_y,'color':sc_c1},
                          display_names=False)

mark_predicted_line = bqp.Lines(x=X_line,
                               y=y_line,
                               scales={'x':scale_x,'y':scale_y},
                               colors=['white'])

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='X label')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='Y Label')

# Create Figure
figure = bqp.Figure(marks=[mark_scatter,mark_predicted_line],
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    title='Title of plot',
                    title_style={'font-size': '22px'},
                    padding_x=0.05,
                    padding_y=0.05,
                    layout={'width': '100%', 'height': '500px'})

def update_visualization(dataframe):
    """This function will update the plot when provided a new dataframe."""

    # Update Bins() mark
    mark_scatter.x = dataframe['Column 1']
    mark_scatter.y = dataframe['Column 2']

# Display the figure
figure


Figure(animation_duration=500, axes=[Axis(label='X label', scale=LinearScale()), Axis(label='Y Label', orienta…